In [1]:
# Baseline-only evaluation (no trained model comparison)
# Evaluate SINR + metrics over a GU trajectory and visualize results.

from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

from baseline_utils import FixedUAVBaseline, RandomUAVBaseline, resolve_path, resolve_cfg_paths_for_sionna, eval_baseline_over_traj, load_json

ROOT = Path("~/Projects/metaRL_merged").expanduser()
sys.path.append(str(ROOT))  # so "eval_tools" and "outer_loop" are importable

# ---- USER-EDITABLE PATHS ----
GU_TRAJ_CSV = ROOT / "eval_tools/gu_trajectories/gus_traj_raleigh_T50.csv"
MODEL_A_CFG_JSON = ROOT / "inner_loop/model_a/configs/raleigh_modelA.json"

# Optional output folder
OUT_DIR = ROOT / "outer_loop" / "outputs_baselines"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("GU_TRAJ_CSV:", GU_TRAJ_CSV)
print("MODEL_A_CFG_JSON:", MODEL_A_CFG_JSON)
print("OUT_DIR:", OUT_DIR)

GU_TRAJ_CSV: /home/jhuang52/Projects/metaRL_merged/eval_tools/gu_trajectories/gus_traj_raleigh_T50.csv
MODEL_A_CFG_JSON: /home/jhuang52/Projects/metaRL_merged/inner_loop/model_a/configs/raleigh_modelA.json
OUT_DIR: /home/jhuang52/Projects/metaRL_merged/outer_loop/outputs_baselines


In [2]:
cfg = load_json(MODEL_A_CFG_JSON)
cfg = resolve_cfg_paths_for_sionna(cfg, ROOT, inplace=False)

df_traj = pd.read_csv(GU_TRAJ_CSV)
assert set(["t", "gu_id", "x", "y", "z"]).issubset(df_traj.columns), df_traj.columns

t_min = int(df_traj["t"].min())
t_max = int(df_traj["t"].max())
print("t range:", t_min, "->", t_max)

# group trajectory by t
traj_by_t = {int(t): g[["x","y","z"]].to_numpy(dtype=np.float32) for t, g in df_traj.groupby("t")}
n_gus = traj_by_t[t_min].shape[0]
print("n_gus:", n_gus)

# UAV params
n_uav = int(cfg["uav"]["n_uav"])
init_xyz = np.asarray(cfg["uav"]["init_xyz"], dtype=np.float32)
bounds = np.asarray(cfg["uav"]["bounds"], dtype=np.float32)

print("n_uav:", n_uav)
print("init_xyz shape:", init_xyz.shape)
print("bounds:", bounds)


t range: 0 -> 49
n_gus: 150
n_uav: 5
init_xyz shape: (5, 3)
bounds: [[-1000.  1000.]
 [-1000.  1000.]
 [   50.   200.]]


In [3]:
baseline_static = FixedUAVBaseline(init_xyz=init_xyz)
baseline_random = RandomUAVBaseline(bounds=bounds, n_uav=n_uav, seed=0)

print("Baselines ready:",
      baseline_static.__class__.__name__)

print("Baselines ready:",
      baseline_random.__class__.__name__)

Baselines ready: FixedUAVBaseline
Baselines ready: RandomUAVBaseline


In [4]:
t_start, t_end = t_min, t_max

# NOTE: num_samples=200000 can be heavy. If slow, try 50000 or 20000 first.
df_static = eval_baseline_over_traj(
    cfg=cfg,
    traj_by_t=traj_by_t,
    baseline=baseline_static,
    t_start=t_start,
    t_end=t_end,
    num_samples=50000,
    print_scene_path=True,
    store_uav_positions="json",
)

df_random = eval_baseline_over_traj(
    cfg=cfg,
    traj_by_t=traj_by_t,
    baseline=baseline_random,
    t_start=t_start,
    t_end=t_end,
    num_samples=50000,
    print_scene_path=True,
    store_uav_positions="json",
)

df_all = pd.concat([df_static, df_random], ignore_index=True)
out_csv = OUT_DIR / "baseline_only_raleigh_T50.csv"
df_all.to_csv(out_csv, index=False)

print("Saved:", out_csv)
df_all.head()

[eval_baseline_over_traj] baseline=FixedUAVBaseline
[eval_baseline_over_traj] cfg scene xml: /data/jhuang52/Projects/metaRL_merged/scenarios/raleigh_scene/raleigh_union_square.xml


2026-02-10 01:41:58.333144: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-10 01:41:58.373576: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jhuang52/miniconda3/envs/modelA_sb3/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  wa

[eval_baseline_over_traj] baseline=RandomUAVBaseline
[eval_baseline_over_traj] cfg scene xml: /data/jhuang52/Projects/metaRL_merged/scenarios/raleigh_scene/raleigh_union_square.xml
Saved: /home/jhuang52/Projects/metaRL_merged/outer_loop/outputs_baselines/baseline_only_raleigh_T50.csv


,t,baseline,uav_xyz,coverage_tau_db,coverage_count,coverage_ratio,best_sinr_mean,best_sinr_min,best_sinr_max,load_var,per_uav_load_min,per_uav_load_mean,per_uav_load_max
0,0,FixedUAVBaseline,"[[0.0, 0.0, 100.0], [500.0, 0.0, 100.0], [0.0,...",5.0,35.0,0.233333,-74.180153,-120.000008,83.540184,1626.244995,3.0,29.200001,120.0
1,1,FixedUAVBaseline,"[[0.0, 0.0, 100.0], [500.0, 0.0, 100.0], [0.0,...",5.0,34.0,0.226667,-75.098190,-120.000008,83.380524,1662.530518,1.0,29.000000,121.0
2,2,FixedUAVBaseline,"[[0.0, 0.0, 100.0], [500.0, 0.0, 100.0], [0.0,...",5.0,31.0,0.206667,-79.186371,-120.000008,81.123062,1625.101807,3.0,29.000000,120.0
3,3,FixedUAVBaseline,"[[0.0, 0.0, 100.0], [500.0, 0.0, 100.0], [0.0,...",5.0,33.0,0.220000,-78.522995,-120.000008,82.092834,1665.387573,3.0,29.799999,121.0
4,4,FixedUAVBaseline,"[[0.0, 0.0, 100.0], [500.0, 0.0, 100.0], [0.0,...",5.0,40.0,0.266667,-68.475525,-120.000008,81.034668,1480.244995,1.0,28.600000,115.0


---

In [5]:
# --- 2D SINR map settings ---

# Choose which baseline to visualize
BASELINE = baseline_static   # or: baseline_random

# Time steps to render
K = 10
T_LIST = list(range(t_start, min(t_start + K, t_end + 1)))

# Grid settings (trade-off: resolution vs speed)
X_MIN, X_MAX = -1000.0, 1000.0
Y_MIN, Y_MAX = -1000.0, 1000.0
NX, NY = 50, 50   # try 80x80 first if slow

# SINR computation settings (trade-off: accuracy vs speed)
NUM_SAMPLES = 5000   # try 20000 if slow
MAX_DEPTH = 1
CELL_SIZE = (40.0, 40.0)  # larger cell_size can speed up some pipelines
VERBOSE = False

# Receiver height for the 2D map (use GU height if provided)
Z_RX = float(cfg.get("gu", {}).get("height", 1.5))

print("Baseline:", BASELINE.__class__.__name__)
print("T_LIST:", T_LIST)
print("Grid:", NX, "x", NY, "Z_RX:", Z_RX)


Baseline: FixedUAVBaseline
T_LIST: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Grid: 50 x 50 Z_RX: 1.5


In [6]:
import numpy as np
from eval_tools.model_a.model_a_functions import compute_sinr_db_gu_tx

def make_xy_grid(xmin, xmax, ymin, ymax, nx, ny, eps=1e-3):
    # Shrink the upper bounds slightly to stay strictly inside radiomap bounds
    xs = np.linspace(xmin, xmax - eps, nx, dtype=np.float32)
    ys = np.linspace(ymin, ymax - eps, ny, dtype=np.float32)
    X, Y = np.meshgrid(xs, ys)  # shapes (ny, nx)
    return xs, ys, X, Y


xs, ys, X, Y = make_xy_grid(X_MIN, X_MAX, Y_MIN, Y_MAX, NX, NY)

# Flatten into (N, 3) points for "virtual GUs"
gu_xyz_grid = np.stack([X.reshape(-1), Y.reshape(-1), np.full(X.size, Z_RX, dtype=np.float32)], axis=1)

print("gu_xyz_grid shape:", gu_xyz_grid.shape)

def compute_sinr_map_db(cfg, uav_xyz, gu_xyz_grid, ny, nx, *, num_samples, max_depth, cell_size, verbose):
    sinr_db = compute_sinr_db_gu_tx(
        cfg=cfg,
        uav_xyz=uav_xyz,
        gu_xyz=gu_xyz_grid,
        max_depth=max_depth,
        num_samples=num_samples,
        cell_size=cell_size,
        verbose=verbose,
    )
    # sinr_db is shape (N,) -> reshape to (ny, nx)
    sinr_map = np.asarray(sinr_db, dtype=np.float32).reshape(ny, nx)
    return sinr_map


gu_xyz_grid shape: (2500, 3)


In [7]:
# import matplotlib.pyplot as plt

# t0 = T_LIST[0]
# uav_xyz = BASELINE.get_uav_xyz(t0)

# sinr_map_db = compute_sinr_map_db(
#     cfg, uav_xyz, gu_xyz_grid, NY, NX,
#     num_samples=NUM_SAMPLES, max_depth=MAX_DEPTH, cell_size=CELL_SIZE, verbose=VERBOSE
# )

# plt.figure()
# plt.imshow(
#     sinr_map_db,
#     origin="lower",
#     extent=[X_MIN, X_MAX, Y_MIN, Y_MAX],
#     aspect="equal",
# )
# plt.colorbar(label="SINR (dB)")
# plt.title(f"SINR map (dB) | t={t0} | {BASELINE.__class__.__name__}")
# plt.xlabel("x")
# plt.ylabel("y")

# # Optional: overlay UAV positions
# plt.scatter(uav_xyz[:, 0], uav_xyz[:, 1], marker="x")
# plt.show()
